In [52]:
#check the efficiency on different tensors cigkl stiffness tensor, sometimes the order of the index affect the efficiency of the compressness (didnt do prrLU on it) maybe due to some other TCI method choice?
prrlu_complete_pivot_LDU = prrlu_complete_pivot_LDU_direct
import itertools
import random
import numpy as np
from dataclasses import dataclass
from typing import Callable, List, Sequence, Tuple, Optional

Array = np.ndarray


# ----------------------------
# TT core utilities
# ----------------------------

def tt_random_cores(dims: Sequence[int], ranks: Sequence[int], seed: int = 0) -> List[Array]:
    """
    Create random TT cores with shapes (r_{k-1}, n_k, r_k).
    ranks must have length= number of legs+1 with ranks[0]=ranks[-1]=1.
    """
    rng = np.random.default_rng(seed)
    N = len(dims)
    assert len(ranks) == N + 1 #Number of ranks (since the first core need 0th rank as well)
    assert ranks[0] == 1 and ranks[-1] == 1
    cores = []
    for k, n in enumerate(dims): #enumerate loop through each leg automatically and create N cores
        rL, rR = ranks[k], ranks[k+1]
        cores.append(rng.standard_normal((rL, n, rR))) #the 3D array core
    return cores


def tt_eval_entry(cores: List[Array], index: Sequence[int]) -> float:
    """
    Evaluate a single tensor entry given TT cores at a multi-index.
    index length must equal number of cores.
    """
    N = len(cores)
    assert len(index) == N
    v = cores[0][0, index[0], :]          # (r1,) v is the current dimension of the matrix slice multiplication, equal to the rank of current SVD matrix
    for k in range(1, N-1):
        v = v @ cores[k][:, index[k], :]  # from kth core we fix index of the input sigma_k and get 2D slice (r_{k+1},)
    v = v @ cores[-1][:, index[-1], 0]    # scalar
    return float(v)


def tt_contract_all(cores: List[Array]) -> Array:
    """
    Materialize full tensor from TT cores (only for small dims).
    Returns ndarray with shape dims.
    """
    N = len(cores)
    dims = [G.shape[1] for G in cores]
    # Start with first core: shape (n1, r1)
    X = cores[0][0, :, :]  # (n1, r1)
    for k in range(1, N):
        G = cores[k]  # (rL, nk, rR)
        # Contract X (..., rL) with G (rL, nk, rR) -> (..., nk, rR)
        X = np.tensordot(X, G, axes=([-1], [0]))
        # Now X shape: (*prev_dims, nk, rR)
    # Final rank should be 1
    assert X.shape[-1] == 1
    return np.squeeze(X, axis=-1)  # shape dims



# ----------------------------
# Tensor Oracle interface
# ----------------------------

@dataclass
@dataclass
class TensorOracle:
    dims: Tuple[int, ...] #store dim as tuple of integers
    entry: Callable[[Tuple[int, ...]], float]
    n_calls: int = 0

    def __call__(self, idx: Sequence[int]) -> float:
        self.n_calls += 1
        idx = tuple(int(i) for i in idx) # convert the input indices for each leg to tuple for oracle to evaluate
        return self.entry(idx)

    def reset_counter(self):
        self.n_calls = 0


    def block(self,
              I_left: Sequence[Tuple[int, ...]],
              S_mid: Sequence[Tuple[int, ...]],
              J_right: Sequence[Tuple[int, ...]]) -> Array:
        """
        Evaluate a sampled block:
          F(I_left, S_mid, J_right)
        where each element of I_left and J_right is a multi-index prefix/suffix.

        - I_left: list of length m, each a tuple of length (k-1)
        - S_mid : list of physical indices tuples for the middle modes (usually 1 or 2 legs)
        - J_right: list of length p, each a tuple of length (N-k) etc.

        Returns array of shape (m, |S_mid|, p)
        """
        m, s, p = len(I_left), len(S_mid), len(J_right)
        out = np.empty((m, s, p), dtype=float) #create a 3D array of such ashape
        for a, il in enumerate(I_left):
            for b, sm in enumerate(S_mid):
                for c, jr in enumerate(J_right):
                    idx = il + sm + jr
                    out[a, b, c] = self.entry(idx)   #generate all combinations of indices at (a,b,c) with indice sets (il, sm, jr)
        return out


def oracle_from_tt(cores: List[Array]) -> TensorOracle:
    dims = tuple(G.shape[1] for G in cores) # collect second dimension of each core=dimension of each leg into a tuple
    def _entry(idx: Tuple[int, ...]) -> float:
        return tt_eval_entry(cores, idx) # takes a list/tuple of indices as input and returns the tensor entry value ny calling tt_eval
    return TensorOracle(dims=dims, entry=_entry)


# ----------------------------
# Quick sanity test
# ----------------------------

dims = (3, 3, 3, 3)
ranks = (1, 2, 2, 2, 1)  # TT ranks
gt_cores = tt_random_cores(dims, ranks, seed=42)
F = oracle_from_tt(gt_cores)

# test a random entry
print("F(0,1,2,0) =", F((0,1,2,0)))

# materialize full tensor for small dims and compare
full = tt_contract_all(gt_cores)
print("Full tensor shape:", full.shape)
print("Check same entry:", full[0,1,2,0])
max_diff = 0.0
for _ in range(20):
    idx = tuple(np.random.randint(0, d) for d in dims)
    diff = abs(F(idx) - full[idx])
    max_diff = max(max_diff, diff)

print("Max abs diff (20 samples):", max_diff)

F(0,1,2,0) = -1.0102064154018084
Full tensor shape: (3, 3, 3, 3)
Check same entry: -1.0102064154018084
Max abs diff (20 samples): 2.7755575615628914e-16


Testing to see if the error is around 0 for TT with random matrix-core to mimic SVD

In [54]:
def prrlu_complete_pivot_LDU_direct(
    A: np.ndarray,
    rmax: int,
    eps: float = 1e-12,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]: #need 7 outputs, row col indices, row permutation matrix and col premutation matix, L,D and U 
    """
    Complete-pivoting rank-revealing LU, returned explicitly as L D U1.

    Produces:
        Pr @ A @ Pc ≈ L @ D @ U1

    Shapes:
        L  : (m, r)  unit-lower (rectangular)
        D  : (r, r)  diagonal (pivots)
        U1 : (r, n)  unit-upper in first r columns (rectangular)
    """
    Awork = np.array(A, dtype=float, copy=True) # do a copy of the input matrix so we don't modify it
    m, n = Awork.shape
    rmax = min(rmax, m, n)

    prow = np.arange(m) #tracks the current ordering of rows, expressed in original row labels, updated whenever rows are swapped
    pcol = np.arange(n) # same for cols 

    piv_rows = []
    piv_cols = []

    pivot0 = None
    r = 0 #prrLU steps, also the rank
    # We'll store pivots (diagonal D) as a vector first, then make diag matrix.
    d_list = []
    
    for k in range(rmax):
        # Find largest pivot in submatrix starting at row/col k
        sub = np.abs(Awork[k:, k:])
        i_rel, j_rel = np.unravel_index(np.argmax(sub), sub.shape) #argmax to find largest element in sub matrix and use unravel to find its index
        i = k + i_rel #true indices in the big matrix 
        j = k + j_rel
        piv = Awork[i, j] #read the element value at pivot since we need it as stopping criterion and to vide the col/row by 

        if pivot0 is None:
            pivot0 = abs(piv) if abs(piv) != 0 else 1.0 #Store scale as the magnitude of the first pivot. If the first pivot is zero (degenerate), use 1.0 to avoid multiplying by zero later.

        if abs(piv) <= eps * pivot0:
            break #stop if the remaining pivot is tiny compared to matrix scale 

        # Swap rows/cols to move pivot to (k,k)
        if i != k:                                  #if i is not equal to k
            Awork[[k, i], :] = Awork[[i, k], :]     #for kth and ith row and all columns
            prow[[k, i]] = prow[[i, k]]             # swap i to k 
        if j != k:
            Awork[:, [k, j]] = Awork[:, [j, k]]
            pcol[[k, j]] = pcol[[j, k]]

        piv_rows.append(prow[k])
        piv_cols.append(pcol[k]) #pivot is now at (k,k) tge true original indices are prow[k] and pcol[k]

        pivot = Awork[k, k]
        if pivot == 0:
            break

        d_list.append(pivot) #add the pivot element to the list of ds and later turn it into matrix D 

        # 3) compute L multipliers below pivot: L[i,k] = A[i,k] / pivot
        if k + 1 < m:
            Awork[k+1:, k] /= pivot # for rows k+1 to the end and col k we divide by pivot to later store in L matrix 

        # 4) normalize pivot row to become U1 row:
        #    U1[k, j] = A[k, j] / pivot, for j >= k
        Awork[k, k:] /= pivot #same for U matrix but all cols
        # Now Awork[k,k] should be 1 (unit diagonal for U1)
        Awork[k, k] = 1.0

        # 5) Schur complement update:
        #    A22 -= L21 * (Dk * U1_12)
        # Here:
        #   L21 is Awork[k+1:, k]
        #   U1_12 is Awork[k, k+1:]
        #   Dk is pivot
        if (k + 1 < m) and (k + 1 < n): #not at the last row or col
            Awork[k+1:, k+1:] -= np.outer(Awork[k+1:, k], pivot * Awork[k, k+1:])#left is matrix A_22 the schur complement right is L21-U12, basically the formula for schur update A/[A_11]=(lower trig of L)D(upper trig of U)

        r += 1

    # Build outputs
    piv_rows = np.array(piv_rows, dtype=int)
    piv_cols = np.array(piv_cols, dtype=int)

    # Permutation matrices (debugging)
    Pr = np.eye(m)[np.argsort(prow)]
    Pc = np.eye(n)[:, np.argsort(pcol)] #check to see if LDU is equal to Pr A Pc

    if r == 0:
        L = np.zeros((m, 0))
        D = np.zeros((0, 0))
        U1 = np.zeros((0, n))
        return piv_rows, piv_cols, Pr, Pc, L, D, U1 #prrLU failed

    # Extract L (unit-lower) from Awork: below diag in first r columns + identity
    L = np.tril(Awork[:, :r], k=-1) + np.eye(m, r)

    # Extract U1 (unit-upper) from Awork: first r rows, upper-triangular-ish
    # Since we normalized pivot rows, Awork already stores U1 in those rows.
    U1 = np.triu(Awork[:r, :], k=0)

    # Build D
    D = np.diag(np.array(d_list, dtype=float))

    return piv_rows, piv_cols, Pr, Pc, L, D, U1


# ----------------------------
# Quick test
# ----------------------------
rng = np.random.default_rng(0)
A = rng.integers(-5, 6, size=(10, 12)).astype(float)

piv_r, piv_c, Pr, Pc, L, D, U1 = prrlu_complete_pivot_LDU_direct(A, rmax=6, eps=1e-12)

A_perm = Pr @ A @ Pc
rel_err = np.linalg.norm(A_perm - L @ D @ U1) / np.linalg.norm(A_perm)
print("r =", len(piv_r))
print("diag(D) =", np.diag(D) if D.size else D)
print("relative error =", rel_err)

if D.size:
    print("diag(U1[:, :r]) =", np.diag(U1[:, :len(piv_r)]))

def skeleton_approx_from_pivots(A: np.ndarray, piv_rows: np.ndarray, piv_cols: np.ndarray, reg: float = 0.0):
    """
    Given pivot rows/cols, form a CI approximation:
        A ≈ A[:,J] @ inv(A[I,J]) @ A[I,:]

    Returns:
        A_hat
    """
    I, J = piv_rows, piv_cols
    C = A[:, J]                 # m x r
    R = A[I, :]                 # r x n
    P = A[np.ix_(I, J)]         # r x r

    if reg > 0:
        P = P + reg * np.eye(P.shape[0])
    X = np.linalg.solve(P, R)   # r x n
    return C @ X

# ----------------------------
# Quick test of CI on a random matrix
# ----------------------------
rng = np.random.default_rng(0)
A = rng.integers(-5, 6, size=(30, 40)).astype(float)


piv_r, piv_c, Pr, Pc, L, D, U1 = prrlu_complete_pivot_LDU(A, rmax=4, eps=1e-12)


print("Selected rank r =", len(piv_r))
print("First few pivot rows:", piv_r[:5])
print("First few pivot cols:", piv_c[:5])

# Check reconstruction quality of permuted LU: Pr*A*Pc ≈ L*U
A_perm = Pr @ A @ Pc
if len(piv_r) > 0:
    rel_err = np.linalg.norm(A_perm - L @ D @ U1) / np.linalg.norm(A_perm)
    print("Relative LU factorization error:", rel_err)

# Check skeleton approximation error using the same pivots       cigkl stiffness tensor
if len(piv_r) > 0:
    A_hat = skeleton_approx_from_pivots(A, piv_r, piv_c)
    rel_err_skel = np.linalg.norm(A - A_hat) / np.linalg.norm(A) #entry difference then do root mean square and normalised
    print("Relative skeleton approx error:", rel_err_skel)

r = 6
diag(D) = [-5.         -9.         -7.55555556  9.55882353 -7.404      -6.97855629]
relative error = 1.1192760593717226
diag(U1[:, :r]) = [1. 1. 1. 1. 1. 1.]
Selected rank r = 4
First few pivot rows: [ 0  3 14 22]
First few pivot cols: [ 5 23 32 12]
Relative LU factorization error: 1.1517735944386922
Relative skeleton approx error: 1.077389632898274


Logic: prrLU still requires the detail of all entries in a matrix therefore we choose sub matrix from the full matrix first and then apply prrLU
Testing: use random matrix to test the accuracy of algorithm just in case if the implimentation is not bugged

In [60]:

MultiIndex = Tuple[int, ...]


def decode_row_id(row_id: int, I_left: List[MultiIndex], n_k: int) -> MultiIndex:
    """
    To get the index of I after prrLU
    Row index in the reshaped 2-site matrix corresponds to (i_{k-1}, sigma_k).

    We pack rows as:
        row_id = a * n_k + sigma_k 
    where:
        a is index into I_left
        sigma_k in {0,...,n_k-1} a

    Returns the new left multi-index i_k = (i_{k-1}, sigma_k).
    """
    a = row_id // n_k
    """
    // used for integer division
    %for the remainder
    """
    sigma_k = row_id % n_k
    return I_left[a] + (sigma_k,)


def decode_col_id(col_id: int, J_right: List[MultiIndex], n_kp1: int) -> MultiIndex:
    """
    Column index corresponds to (sigma_{k+1}, j_{k+2}).

    We pack cols as:
        col_id = sigma_{k+1} * |J_right| + b
        (swap the ordering copared to row id, since for row id the sigma is on the right side and for col id sigma is on the left
    where:
        b is index into J_right
        sigma_{k+1} in {0,...,n_{k+1}-1}

    Returns new right multi-index j_{k+1} = (sigma_{k+1}, j_{k+2}).
    """
    R = len(J_right)
    sigma_kp1 = col_id // R
    b = col_id % R
    return (sigma_kp1,) + J_right[b]


def two_site_block_matrix(F: TensorOracle, I_left: List[MultiIndex], k: int, J_right: List[MultiIndex]) -> np.ndarray:
    """
    Build the 2-site sampled block:
        Π_k = F(I_{k-1}, sigma_k, sigma_{k+1}, J_{k+2})

    where:
      - I_left contains multi-indices of length k-1
      - J_right contains multi-indices of length N-(k+1)
      - k is 1-based site index in math; here we'll use 0-based Python index for cores.

    Returns Π as a 2D matrix with shape:
      rows = |I_left| * n_k
      cols = n_{k+1} * |J_right|
    """
    N = len(F.dims)
    assert 0 <= k <= N-2
    """
    sites are 0..N-1
      this function builds the block for sites k and k+1 (so requires k <= N-2)
      """
    n_k = F.dims[k]
    n_kp1 = F.dims[k+1]

    # Build lists of middle index tuples for the pair of the two free legs, same as two for loops
    S_mid = [(s1, s2) for s1 in range(n_k) for s2 in range(n_kp1)]  # length n_k*n_{k+1}

    # Query the oracle for the 3D block of shape (|I_left|, n_k*n_{k+1}, |J_right|)
    block3 = F.block(I_left=I_left, S_mid=S_mid, J_right=J_right)

    # Now reshape into a matrix with:
    #   rows = |I_left| * n_k
    #   cols = n_{k+1} * |J_right|
    #
    # block3 is indexed by:
    #   block3[a, b, c]
    # where b corresponds to (sigma_k, sigma_{k+1}) in S_mid order.
    #
    # We'll reshape by first reshaping the middle axis into (n_k, n_{k+1}):
    m = len(I_left)
    p = len(J_right)
    block4 = block3.reshape(m, n_k, n_kp1, p)          # (m, n_k, n_{k+1}, p)
    A = block4.reshape(m * n_k, n_kp1 * p)             # (m*n_k, n_{k+1}*p)
    return A # The 2D matrix for prrLU


# ----------------------------
# Demo: one local 2-site update
# ----------------------------

def random_multi_indices(dims, r):
    all_idx = list(itertools.product(*[range(d) for d in dims]))
    random.shuffle(all_idx)
    return all_idx[:r]

N = len(F.dims)
k = 1  # build block for sites 1 and 2 (0-based)

# Initialize very small pivot sets (path-based):

# For this local block, I_left should be length k (i.e. k sites on the left of site k since we start at k = 0)
# But in math it is I_{k-1}. For k=1 (sites 1&2), k-1=0 so I_left is empty prefix.
# for any k, we store multi-indices of length k (sites 0..k-1)
# and J_right of length N-(k+2) (sites k+2..N-1)
#
# We'll start with 3 random indices
I_left  = random_multi_indices(F.dims[:k], r=3) #first k tensor dimensions with three realisations (of 0,1,2) doesnt include kth 
J_right = random_multi_indices(F.dims[k+2:], r=3) #k+2th index to the end (include k+2th), also three realisations


F.reset_counter()
A = two_site_block_matrix(F, I_left, k, J_right) #construct sample matrix with I and J while loop over indices in two free legs

print("Built 2-site block matrix A with shape:", A.shape)
print("Oracle calls used to build A:", F.n_calls)
print("\n=== Decoding BEFORE pivot update ===")

n_k = F.dims[k]
n_kp1 = F.dims[k+1]

print("\nRow ID → tensor multi-index (prefix, σ_k):")
for rid in range(A.shape[0]):
    decoded = decode_row_id(rid, I_left, n_k)
    print(f"row {rid:2d} -> {decoded}")

print("\nCol ID → tensor multi-index (σ_{k+1}, suffix):")
for cid in range(A.shape[1]):
    decoded = decode_col_id(cid, J_right, n_kp1)
    print(f"col {cid:2d} -> {decoded}")

# Run prrLU on this small matrix
piv_r, piv_c, Pr, Pc, L, D, U1 = prrlu_complete_pivot_LDU(A, rmax=4, eps=1e-12) 

print("Local discovered rank r =", len(piv_r))
print("Pivot row ids (in A):", piv_r)
print("Pivot col ids (in A):", piv_c)

# Decode pivot ids into NEW multi-index sets
n_k = F.dims[k]
n_kp1 = F.dims[k+1]

I_k = [decode_row_id(rid, I_left, n_k) for rid in piv_r]          # multi-indices length k+1
J_kp1 = [decode_col_id(cid, J_right, n_kp1) for cid in piv_c]     # multi-indices length N-(k+1)

print("Updated I_k (prefix indices up to site k):", I_k)
print("Updated J_{k+1} (suffix indices from site k+1):", J_kp1)


Built 2-site block matrix A with shape: (9, 9)
Oracle calls used to build A: 0

=== Decoding BEFORE pivot update ===

Row ID → tensor multi-index (prefix, σ_k):
row  0 -> (0, 0)
row  1 -> (0, 1)
row  2 -> (0, 2)
row  3 -> (2, 0)
row  4 -> (2, 1)
row  5 -> (2, 2)
row  6 -> (1, 0)
row  7 -> (1, 1)
row  8 -> (1, 2)

Col ID → tensor multi-index (σ_{k+1}, suffix):
col  0 -> (0, 0)
col  1 -> (0, 2)
col  2 -> (0, 1)
col  3 -> (1, 0)
col  4 -> (1, 2)
col  5 -> (1, 1)
col  6 -> (2, 0)
col  7 -> (2, 2)
col  8 -> (2, 1)
Local discovered rank r = 2
Pivot row ids (in A): [5 4]
Pivot col ids (in A): [6 0]
Updated I_k (prefix indices up to site k): [(2, 2), (2, 1)]
Updated J_{k+1} (suffix indices from site k+1): [(2, 0), (0, 0)]
